<a href="https://colab.research.google.com/github/Adil1979/P6_Tag_Recommendation_Stackoverflow/blob/master/Unsupervised_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projet6: Catégorisation automatique de questions**

In [84]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.lines import Line2D
import matplotlib as mpl
#import chart_studio.plotly.plotly as py
from collections import Counter
#import plotly.plotly as py
import seaborn as sns
import datetime as dt
import calendar as cld
from scipy import stats
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from os import listdir
import glob
import missingno as msno
#from feature_engine.categorical_encoders import OneHotCategoricalEncoder
from sklearn.decomposition import PCA
from matplotlib import dates
import nltk
import re
from nltk.corpus import stopwords 
nltk.download('stopwords')
from nltk.stem.porter import PorterStemmer 
import string
from collections import Counter
from bs4 import BeautifulSoup
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
random_state = 747

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [91]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 5.1MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=b2150220b3fbc308b9f1662df0cf171d9359e939dba63d6313b92debe0d12f76
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis


In [85]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [86]:
questions = pd.read_csv('/content/drive/My Drive/CentraleSupelec_OCR/P6/questions1.csv')
questions = questions.drop('Unnamed: 0', axis=1)
questions

,Title_Body
0,static
1,function function struct
2,mongodb object mongodb api date mongodb object...
3,laravel laravel post post text post post text ...
4,function keras keras input module keras keras ...
...,...
39995,multithreading c c c++
39996,class python class python nuget r c module ass...
39997,angular jasmine testing jasmine testing jasmin...
39998,object python class object c c c c object


In [88]:
questions['Title_Body'] = questions['Title_Body'].apply(lambda text: str(text).split())

In [89]:
questions = questions['Title_Body'] 
questions.head(10)

0                                             [static]
1                         [function, function, struct]
2    [mongodb, object, mongodb, api, date, mongodb,...
3    [laravel, laravel, post, post, text, post, pos...
4    [function, keras, keras, input, module, keras,...
5                       [function, function, function]
6    [dataframe, pandas, pandas, dataframe, date, c...
7    [list, random, random, random, random, filter,...
8                                         [java, java]
9    [tensorflow, pandas, tensorflow, numpy, seabor...
Name: Title_Body, dtype: object

In [74]:
from gensim import corpora
dictionary = corpora.Dictionary(questions)
corpus = [dictionary.doc2bow(text) for text in questions]

In [76]:
len(dictionary)

325

In [77]:
corpus

[[(0, 1)],
 [(1, 2), (2, 1)],
 [(3, 1), (4, 1), (5, 4), (6, 2)],
 [(7, 1), (8, 2), (9, 6), (10, 3)],
 [(1, 3), (6, 1), (7, 1), (11, 5), (12, 4)],
 [(1, 3)],
 [(4, 4), (13, 2), (14, 2), (15, 2)],
 [(1, 8), (6, 2), (10, 1), (16, 18), (17, 1), (18, 9), (19, 1), (20, 14)],
 [(21, 2)],
 [(7, 23),
  (12, 1),
  (13, 12),
  (14, 2),
  (15, 2),
  (19, 1),
  (20, 1),
  (22, 1),
  (23, 2),
  (24, 1),
  (25, 2),
  (26, 1),
  (27, 4),
  (28, 5),
  (29, 1),
  (30, 2),
  (31, 2),
  (32, 24),
  (33, 1),
  (34, 1),
  (35, 14),
  (36, 32)],
 [(37, 1), (38, 2), (39, 2), (40, 3)],
 [(41, 1)],
 [(10, 1), (16, 3), (18, 1), (42, 2), (43, 1), (44, 1), (45, 1)],
 [(1, 3),
  (12, 1),
  (13, 9),
  (19, 2),
  (30, 2),
  (32, 7),
  (38, 67),
  (46, 72),
  (47, 11),
  (48, 1),
  (49, 2),
  (50, 9),
  (51, 8),
  (52, 11),
  (53, 1),
  (54, 1),
  (55, 2),
  (56, 1),
  (57, 8),
  (58, 2),
  (59, 4),
  (60, 59),
  (61, 1),
  (62, 4)],
 [(7, 3), (41, 3), (46, 2), (63, 10)],
 [(6, 6), (10, 2), (18, 4), (33, 3), (50, 1), 

In [78]:
lda = gensim.models.ldamodel.LdaModel

In [80]:
ldamodel = lda(corpus, num_topics=5, id2word = dictionary, passes=50)

In [82]:
print("LDA model is: \n", ldamodel.print_topics(num_topics=5, num_words=5))

LDA model is: 
 [(0, '0.178*"file" + 0.139*"c" + 0.057*"build" + 0.056*"python" + 0.048*"module"'), (1, '0.220*"java" + 0.077*"api" + 0.071*"server" + 0.047*"angular" + 0.044*"web"'), (2, '0.237*"android" + 0.209*"class" + 0.071*"html" + 0.041*"firebase" + 0.035*"text"'), (3, '0.156*"string" + 0.128*"function" + 0.098*"list" + 0.072*"class" + 0.069*"object"'), (4, '0.106*"text" + 0.106*"input" + 0.104*"image" + 0.069*"date" + 0.034*"dataframe"')]
